In [ ]:
import os
import sys

# Heritage Forest
180 Sqm
630 saplings
26 species

**Canopy 10% 63**
Lime - small 21
Oak Sessile 21
Oak English 21

**Tree 30% 189**
Hornbeam 34
Willow goat 26
Willow bay 26 Rowan 26
Maple 26
Crab apple 26
Willow - grey 26

**Sub Tree 30% -189**
Yew 26
Holly 26
Hazel 34
Hawthorn 26
Blackthorn 26
Whitebeam 26
Willow wooly 26

**Shrub 30% 189**
Box 21
Elder 21
Alder buckthorn 21
Dogwood 21
Spindle 21
Wild privet 21
Dog rose 21
Guelder rose 21
Filed rose 21

In [ ]:
tree_families = {
    'Lime - small': 'Deciduous',
    'Oak Sessile': 'Deciduous',
    'Oak English': 'Deciduous',
    'Hornbeam': 'Deciduous',
    'Willow goat': 'Deciduous',
    'Willow bay': 'Deciduous',
    'Rowan': 'Deciduous',
    'Maple': 'Deciduous',
    'Crab apple': 'Deciduous',
    'Willow - grey': 'Deciduous',
    'Yew': 'Coniferous',
    'Holly': 'Evergreen', # Evergreen but not a conifer
    'Hazel': 'Deciduous',
    'Hawthorn': 'Deciduous',
    'Blackthorn': 'Deciduous',
    'Whitebeam': 'Deciduous',
    'Willow wooly': 'Deciduous'
}